In [1]:
# librerias
from PIL import Image
import sys 
from pdf2image import convert_from_path 
import os, glob, re
from os import listdir
from PyPDF2 import PdfFileWriter, PdfFileReader
import boto3
import pandas as pd
from trp import Document

In [2]:
dirpath = os.getcwd() # directorio actual

# selecciona el pdf de la carpeta
os.chdir("./input_pdf_here")
for file in glob.glob("*.pdf"):
    input_pdf = file
    break
pdf_input_dir = os.getcwd() # directorio actualizado

input_pdf_fullpath = os.path.join(dirpath, "input_pdf_here", input_pdf) # input 
#pdf_output_dir = os.path.join(dirpath, "output_is_here", "output.pdf") # output

# Funciones varias
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']

In [3]:
''' 
Part #1 : Convertir pdf a imagenes 
'''
# Almacenar todas las páginas del PDF en una variable
pages = convert_from_path(input_pdf_fullpath, 500, poppler_path="C:\\Users\\DiegoIgnacioPavezOla\\Desktop\\Investigacion\\OCR SCRIPT FINAL\\poppler-0.68.0\\bin") # PATH POPPLER

# Contador para almacenar imágenes de cada página de PDF a imagen 
image_counter = 1
# Iterar a través de todas las páginas almacenadas anteriormente 
for page in pages: 
    print(f"convertir pdf a jpg en la pagina: {page}")
    # PDF page n -> page_n.jpg 
    filename = "page_"+str(image_counter)+".jpg"
    # Guardar la imagen de la página en el sistema  
    page.save(filename, 'JPEG') 
    # Incrementa el contador para actualizar el nombre del archivo 
    image_counter = image_counter + 1
    print(f"Listo para la pagina: {page}")

convertir pdf a jpg en la pagina: <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4132x5848 at 0x15C4D38C8E0>
Listo para la pagina: <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4132x5848 at 0x15C4D38C8E0>
convertir pdf a jpg en la pagina: <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4132x5848 at 0x15C4D38C970>
Listo para la pagina: <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4132x5848 at 0x15C4D38C970>
convertir pdf a jpg en la pagina: <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4132x5848 at 0x15C64EB29A0>
Listo para la pagina: <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4132x5848 at 0x15C64EB29A0>
convertir pdf a jpg en la pagina: <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4132x5848 at 0x15C64EB2AF0>
Listo para la pagina: <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4132x5848 at 0x15C64EB2AF0>


In [93]:
''' 
Part #2 - Reconocer el texto de las imágenes mediante OCR
'''

original_pages = [] # recordar que la numeración de las páginas empieza por el 0
keep_pages = [] # recordar que la numeración de las páginas empieza por el 0
dataframes = []
tableTitle = []
responseAWS = []

# Variable para obtener el recuento del número total de páginas  
filelimit = image_counter-1

# tesseract consu ruta opcional para usar pytesesract
#pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe' # Tesseract

# Creación de un archivo de texto para escribir la salida
for i in range(1, filelimit + 1):
    print(f"Reconocimiento de imagen para la imagen #:{i}")
    #print(f"Generando el texto de la imgen #:{i}")
    # Documento
    filename = "page_"+str(i)+".jpg"
    #outfile = "page_"+str(i)+".txt"
    #f = open(outfile, "w")

    print("Ingreso a la api de AWS")
    # Amazon Textract
    textract = boto3.client('textract')

    # Llamada a Amazon Textract
    with open(filename, "rb") as document:
        response = textract.analyze_document(
            Document={
                'Bytes': document.read(),
            },
            FeatureTypes=["TABLES"])


    blocks = response['Blocks']
    tables = map_blocks(blocks, 'TABLE')
    cells = map_blocks(blocks, 'CELL')
    words = map_blocks(blocks, 'WORD')
    selections = map_blocks(blocks, 'SELECTION_ELEMENT')


    for table in tables.values():

        # Determinar todas las celdas que pertenecen a esta tabla
        table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

        # Determinar el número de filas y columnas de la tabla
        n_rows = max(cell['RowIndex'] for cell in table_cells)
        n_cols = max(cell['ColumnIndex'] for cell in table_cells)
        content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

        # Rellenar cada celda
        for cell in table_cells:
            cell_contents = [
                words[child_id]['Text']
                if child_id in words
                else selections[child_id]['SelectionStatus']
                for child_id in get_children_ids(cell)
            ]
            i = cell['RowIndex'] - 1
            j = cell['ColumnIndex'] - 1
            content[i][j] = ' '.join(cell_contents)

        # Suponemos que la primera fila corresponde a los nombres de las columnas
        dataframe = pd.DataFrame(content[1:], columns=content[0])
        print("Guardo la tabla como un dataframe en la lista")
        dataframes.append(dataframe)
        responseAWS.append(response)
        for item in response["Blocks"]:
            if item["BlockType"] == "LINE":
                #print ('\033[94m' +  item["Text"] + '\033[0m')
                if (re.match(r"([T]+[\w\w\w\w\w]+[\s]+[\d])", item["Text"])):
                    tableTitle.append(item["Text"])     

Reconocimiento de imagen para la imagen #:1
Ingreso a la api de AWS
Reconocimiento de imagen para la imagen #:2
Ingreso a la api de AWS
Guardo la tabla como un dataframe en la lista
Guardo la tabla como un dataframe en la lista
Guardo la tabla como un dataframe en la lista
Reconocimiento de imagen para la imagen #:3
Ingreso a la api de AWS
Guardo la tabla como un dataframe en la lista
Reconocimiento de imagen para la imagen #:4
Ingreso a la api de AWS


In [94]:
tableTitle

['Table 1: Composition of the drug polymer film and film weight, thickness before and after cross linking',
 'Table 1 shows the composition of the prepared films along',
 'Table 2: Folding endurance, tensile strength and elongation at break of films before and after cross-linking',
 'Table 3: Equilibrium swelling, bioadhesive strength and force of adhesion of films before and after cross-linking',
 'Table 1: Composition of the drug polymer film and film weight, thickness before and after cross linking',
 'Table 1 shows the composition of the prepared films along',
 'Table 2: Folding endurance, tensile strength and elongation at break of films before and after cross-linking',
 'Table 3: Equilibrium swelling, bioadhesive strength and force of adhesion of films before and after cross-linking',
 'Table 1: Composition of the drug polymer film and film weight, thickness before and after cross linking',
 'Table 1 shows the composition of the prepared films along',
 'Table 2: Folding endurance

In [115]:
tableTitle = []
for item in responseAWS[0]["Blocks"]:
    if item["BlockType"] == "LINE":
        #print ('\033[94m' +  item["Text"] + '\033[0m')
        if (re.match(r"([T]+[\w\w\w\w\w]+[\s]+[\d])", item["Text"])):
            tableTitle.append(item["Text"])          

In [116]:
tableTitle

['Table 1: Composition of the drug polymer film and film weight, thickness before and after cross linking',
 'Table 1 shows the composition of the prepared films along',
 'Table 2: Folding endurance, tensile strength and elongation at break of films before and after cross-linking',
 'Table 3: Equilibrium swelling, bioadhesive strength and force of adhesion of films before and after cross-linking']

In [120]:
 responseAWS[0]

{'DocumentMetadata': {'Pages': 1},
 'Blocks': [{'BlockType': 'PAGE',
   'Geometry': {'BoundingBox': {'Width': 0.9997640252113342,
     'Height': 0.9998332858085632,
     'Left': 0.0,
     'Top': 0.0},
    'Polygon': [{'X': 0.0, 'Y': 0.0},
     {'X': 0.9997640252113342, 'Y': 0.0},
     {'X': 0.9997640252113342, 'Y': 0.9998332858085632},
     {'X': 0.0, 'Y': 0.9998332858085632}]},
   'Id': 'c44d04ac-1f2f-41d0-a11d-1529bb01b62c',
   'Relationships': [{'Type': 'CHILD',
     'Ids': ['5f611ab7-ed81-4b11-a8e3-81f45788c283',
      'f6624154-4bc9-4b65-bd14-1934d6ad4b65',
      '17560585-93f8-4724-bf08-9b0c2d1a96be',
      'e8f11c09-eb79-4b1b-91b0-fb52a67fe642',
      '2b626d51-9fe3-4dba-902b-df4f6291b839',
      'b82e4a9c-4998-4fef-ba87-3f5bc61536b4',
      '3d762814-bc66-4ea7-afb5-c68975f48901',
      'e67216fb-21ea-48d1-b550-d7d05c237109',
      '9fb7a4d3-0586-4f1e-a53c-3f2372f5e8cc',
      '8eb65df7-c03e-47c5-b2fd-520d8061777d',
      '3861812b-373b-4195-9c51-c0e69d90b51f',
      '83b0b1c8-9

[]